In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Define the VQ-VAE encoder
def encoder_vqvae(inputs, embedding_dim=64):
    # Apply a series of convolutional layers to encode the input images
    x = tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.Conv2D(embedding_dim, kernel_size=4, strides=2, padding='same')(x)
    return x  # Returns encoded outputs with shape (batch_size, 8, 8, embedding_dim)

# Quantization function to map continuous embeddings to discrete tokens
def quantize(encoded_outputs, embeddings):
    # Reshape the encoded outputs for distance calculation
    flat_outputs = tf.reshape(encoded_outputs, [-1, encoded_outputs.shape[-1]])
    # Calculate distances between the outputs and embeddings
    distances = tf.norm(flat_outputs[:, None] - embeddings[None, :], axis=-1)
    # Get the indices of the closest embeddings
    indices = tf.argmin(distances, axis=-1)
    # Map indices to the quantized embeddings
    quantized = tf.nn.embedding_lookup(embeddings, indices)
    quantized = tf.reshape(quantized, tf.shape(encoded_outputs))  # Reshape to original encoded shape
    return quantized, indices

# Define the VQ-VAE decoder
def decoder_vqvae(quantized):
    # Apply a series of transposed convolutional layers to decode the embeddings
    x = tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu')(quantized)
    x = tf.keras.layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='sigmoid')(x)
    return x  # Returns the reconstructed images

# Initialize embeddings and model input
num_embeddings = 512
embedding_dim = 64
embeddings = tf.Variable(tf.random.normal([num_embeddings, embedding_dim]), name="vqvae_embeddings")
inputs = tf.keras.Input(shape=(32, 32, 3))  # Input shape for CIFAR-10 images

# Encode, quantize, and decode the images
encoded = encoder_vqvae(inputs, embedding_dim)
quantized, token_indices = quantize(encoded, embeddings)
reconstructed = decoder_vqvae(quantized)

# Define the VQ-VAE model
vqvae_model = tf.keras.Model(inputs, reconstructed)
vqvae_model.summary()  # Print the model summary

# Load the pretrained GPT-2 model and tokenizer
transformer_model = TFGPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Helper function to encode tokens for transformer input
def encode_tokens_for_transformer(indices):
    # Flatten the indices and convert them to string representation
    indices_flat = tf.reshape(indices, [-1]).numpy()
    # Convert the indices to a string and encode
    token_ids = tokenizer.encode(" ".join(map(str, indices_flat)), return_tensors="tf")
    return token_ids  # Return token IDs for the transformer

# Load and normalize sample data
(train_images, _), _ = tf.keras.datasets.cifar10.load_data()
train_images = train_images.astype("float32") / 255.0  # Normalize images to [0, 1]

# Training setup
batch_size = 32
epochs = 5
optimizer = tf.keras.optimizers.Adam(1e-3)

# Training loop for VQ-VAE
for epoch in range(epochs):
    for i in range(0, len(train_images), batch_size):
        imgs = train_images[i:i + batch_size]
        with tf.GradientTape() as tape:
            # Forward pass through the VQ-VAE
            encoded_batch = encoder_vqvae(imgs, embedding_dim)
            quantized_batch, indices_batch = quantize(encoded_batch, embeddings)
            recon_imgs = decoder_vqvae(quantized_batch)
            # Calculate reconstruction loss
            reconstruction_loss = tf.reduce_mean(tf.losses.mean_squared_error(imgs, recon_imgs))

        # Backpropagation
        grads = tape.gradient(reconstruction_loss, vqvae_model.trainable_weights + [embeddings])
        optimizer.apply_gradients(zip(grads, vqvae_model.trainable_weights + [embeddings]))

        # Prepare input tokens for the transformer model
        input_tokens = encode_tokens_for_transformer(indices_batch)

        # Ensure that input_tokens has the correct shape for the transformer model
        print(f"Input tokens shape: {input_tokens.shape}")  # Debugging statement

        try:
            # Generate tokens from the transformer model
            generated_sequence = transformer_model.generate(input_tokens, max_length=100)
            print(f"Epoch {epoch+1}, Batch {i // batch_size + 1}, Reconstruction Loss: {reconstruction_loss:.4f}")
        except Exception as e:
            print(f"Error during token generation: {e}")
            continue  # Skip to the next batch on error

# Function to generate a new image from the transformer
def generate_image_from_transformer():
    input_tokens = tf.constant([[transformer_model.config.bos_token_id]])  # Start with BOS token
    generated_sequence = transformer_model.generate(input_tokens, max_length=64)
    indices_generated = [int(idx) for idx in tokenizer.decode(generated_sequence[0]).split()]
    # Reshape the generated indices for quantization
    indices_generated = tf.reshape(indices_generated, (1, 8, 8))
    quantized_generated = tf.nn.embedding_lookup(embeddings, indices_generated)  # Quantize
    generated_image = decoder_vqvae(quantized_generated)  # Decode to get the final image
    return generated_image

# Display the generated image
import matplotlib.pyplot as plt

new_image = generate_image_from_transformer()
plt.imshow(new_image[0])  # Show the generated image
plt.axis('off')  # Turn off axes
plt.show()  # Display the image


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_22 (Conv2D)          (None, 16, 16, 128)          6272      ['input_8[0][0]']             
                                                                                                  
 conv2d_23 (Conv2D)          (None, 8, 8, 64)             131136    ['conv2d_22[0][0]']           
                                                                                                  
 tf.reshape_14 (TFOpLambda)  (None, 64)                   0         ['conv2d_23[0][0]']           
                                                                                            

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (2048 > 1024). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 2048, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing`max_new_tokens`.


Input tokens shape: (1, 2048)
Error during token generation: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -1948 must be >= 0 [Op:Fill] name: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 2048, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing`max_new_tokens`.


Input tokens shape: (1, 2048)
Error during token generation: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -1948 must be >= 0 [Op:Fill] name: 


...

Input tokens shape: (1, 2048)
Error during token generation: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -1948 must be >= 0 [Op:Fill] name: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 2048, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing`max_new_tokens`.


Input tokens shape: (1, 2048)
Error during token generation: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -1948 must be >= 0 [Op:Fill] name: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 2048, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing`max_new_tokens`.


Input tokens shape: (1, 2048)
Error during token generation: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimension -1948 must be >= 0 [Op:Fill] name: 
